In [11]:
from kafka import KafkaProducer
import json
import time
import pandas as pd

df = pd.read_csv('shopping_behavior.csv')

producer = KafkaProducer (bootstrap_servers='35.184.245.41:9092', value_serializer=lambda v: json.dumps(v).encode('utf-8'))

for index, row in df.iterrows():
    row_dict = row.to_dict()

    producer.send('mid_proje', row_dict)
    time.sleep(1)
    print(row_dict)

{'Customer_ID': 1, 'Age': 55, 'Gender': 'Male', 'Item_Purchased': 'Blouse', 'Category': 'Clothing', 'Purchase_Amount': 53, 'Location': 'Kentucky', 'Size': 'L', 'Color': 'Gray', 'Season': 'Winter', 'Review_Rating': 3.1, 'Subscription_Status': 'Yes', 'Shipping_Type': 'Express', 'Discount_Applied': 'Yes', 'Promo_Code_Used': 'Yes', 'Previous_Purchases': 14, 'Payment_Method': 'Venmo', 'Frequency_Purchases': 'Fortnightly'}
{'Customer_ID': 2, 'Age': 19, 'Gender': 'Male', 'Item_Purchased': 'Sweater', 'Category': 'Clothing', 'Purchase_Amount': 64, 'Location': 'Maine', 'Size': 'L', 'Color': 'Maroon', 'Season': 'Winter', 'Review_Rating': 3.1, 'Subscription_Status': 'Yes', 'Shipping_Type': 'Express', 'Discount_Applied': 'Yes', 'Promo_Code_Used': 'Yes', 'Previous_Purchases': 2, 'Payment_Method': 'Cash', 'Frequency_Purchases': 'Fortnightly'}
{'Customer_ID': 3, 'Age': 50, 'Gender': 'Male', 'Item_Purchased': 'Jeans', 'Category': 'Clothing', 'Purchase_Amount': 73, 'Location': 'Massachusetts', 'Size': '

In [ ]:
# pyspark --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 --jars=gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar



from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType ,TimestampType
from pyspark.sql.functions import from_json



spark = SparkSession.builder.appName('kafka').getOrCreate()

buket = "mid-proje"
spark.conf.set('temporaryGcsBucket', buket)
spark.conf.set('parentProject', 'skilled-module-406113')
# {'Customer_ID': 719, 'Age': 22, 'Gender': 'Male', 'Item_Purchased': 'Shirt', 'Category': 'Clothing', 'Purchase_Amount(USD)': 22, 'Location': 'Florida', 'Size': 'M',
#  'Color': 'Peach', 'Season': 'Summer', 'Review_Rating': 4.6, 'Subscription_Status': 'Yes', 'Shipping_Type': 'Standard', 'Discount_Applied': 'Yes', 'Promo_Code_Used': 'Yes', 
# 'Previous_Purchases': 25, 'Payment_Method': 'Bank Transfer', 'Frequency_Purchases': 'Annually'}


schema = StructType([
    StructField('Customer_ID', IntegerType()),
    StructField('Age', IntegerType()),
    StructField('Gender', StringType()),
    StructField('Item_Purchased', StringType()),
    StructField('Category', StringType()),
    StructField('Purchase_Amount', IntegerType()),
    StructField('Location', StringType()),
    StructField('Size', StringType()),
    StructField('Color', StringType()),
    StructField('Season', StringType()),  
    StructField('Review_Rating', FloatType()),
    StructField('Subscription_Status', StringType()),
    StructField('Shipping_Type', StringType()),
    StructField('Discount_Applied', StringType()),
    StructField('Promo_Code_Used', StringType()),
    StructField('Previous_Purchases', IntegerType()), 
    StructField('Payment_Method', StringType()),
    StructField('Frequency_Purchases', StringType())    
])

kafkaDf = spark.readStream.format('kafka') \
            .option('kafka.bootstrap.servers', '35.184.245.41:9092') \
            .option('subscribe', 'mid_proje').load()
# from_json.user_id
# from_json.platform
# data.user_id
# select('data.*') -> user_id, platform, product_id, event, sid, timestamp

read_kafka = kafkaDf.select(from_json(kafkaDf["value"].cast("string"),schema).alias('data')).select('data.*')

# filter_df = read_kafka.filter(read_kafka.event == 'impression')

# read_kafka.writeStream.format('console').start().awaitTermination()

read_kafka.writeStream.format('bigquery') \
    .option('table', 'mid_proje.shopping_behavior') \
    .option('checkpointLocation', '/path/to/checkpoint/dir/in/hdfs') \
    .option("failOnDataLoss", False) \
    .start().awaitTermination()